In [1]:
import os
import sys
from railway import *

In [2]:
# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

Current working directory: 📂 /home/marco/railway-scheduling


In [3]:
# Define problem parameters
N = 10
T = 10
J = 10
P = 2000
K = 3

# Name of the file to load
FILENAME = f"datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# Instantiate the Railway object
rail = Railway.load(FILENAME)
rail.model.setParam('TimeLimit', 60)

# Print the model in summary
print(rail)

Set parameter Username
Set parameter LicenseID to value 2629256
Academic license - for non-commercial use only - expires 2026-02-27
Set parameter TimeLimit to value 60
Railway scheduling problem

Parameters:
N:  10 stations
T:  10 periods
J:  10 jobs
P:  2000 passengers
K:  3 alternative routes
Aj: 10 jobs with arcs
Ja: 9 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   0
Constraints: 0
Objective:   0.0
Status:      LOADED



In [4]:
# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

# Solve the scheduling problem
rail.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 8075 rows, 5000 columns and 22305 nonzeros
Model fingerprint: 0xee19e457
Variable types: 1450 continuous, 3550 integer (3550 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+06]
  Objective range  [1e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+06]
Presolve removed 8037 rows and 4964 columns
Presolve time: 0.01s
Presolved: 38 rows, 36 columns, 108 nonzeros
Variable types: 18 continuous, 18 integer (18 binary)
Found heuristic solution: objective 2282.9119118
Found heuristic solution: objective 2173.5684880
Found heuristic solution: objective 2001.2184410

Root relaxation: objective 1.886618e+03, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes

In [5]:
print(type(rail.S))
print(rail.S)

print()

print(type(rail.Aj))
print(rail.Aj)

print()

print(type(rail.A))
print(rail.A)

<class 'dict'>
{1: 3, 2: 4, 3: 1, 4: 1, 5: 7, 6: 9, 7: 5, 8: 9, 9: 1, 10: 1}

<class 'dict'>
{1: [(1, 7)], 2: [(2, 6)], 3: [(5, 9)], 4: [(1, 6)], 5: [(1, 5)], 6: [(3, 6)], 7: [(6, 10)], 8: [(2, 10)], 9: [(1, 7)], 10: [(1, 10)]}

<class 'list'>
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


In [6]:
S = rail.S.copy()
Aj_original = rail.Aj.copy()  # Keep original reference
Aj = rail.Aj.copy() 
A = rail.A.copy()
Tend = rail.Tend

# Create dictionary of named arcs
An = {}
for i, a in enumerate(A):
    An[f"a{i}"] = a

# Use a regular set instead of OrderedDict
jobs_arc_set = set()

# Convert arcs in Aj dictionary to their names
for job, job_arc_list in Aj_original.items():
    for i, arc in enumerate(job_arc_list):
        # Find the name of the arc in An
        arc_name = [name for name, value in An.items() if value == arc]
        if arc_name:
            # Replace the arc with its name
            Aj[job][i] = arc_name[0]
            
            # Add the arc to the set
            jobs_arc_set.add(arc_name[0])

# Sort the set when you need ordered output
# This sorts by the numerical portion (e.g., "a12" -> 12)
sorted_arc_names = sorted(jobs_arc_set, key=lambda x: int(x[1:]))

# Convert arcs in Aj to their position in sorted_arc_names
for job, job_arc_list in Aj.items():
    for i, arc_name in enumerate(job_arc_list):
        # Find the index of the arc name in sorted_arc_names
        index = sorted_arc_names.index(arc_name)
        # Replace the arc name with its index
        Aj[job][i] = index
        
        
    
print('sorted_arc_names')
print(sorted_arc_names)

print('Aj')
print(Aj)

print('S')
print(S)

sorted_arc_names
['a3', 'a4', 'a5', 'a8', 'a12', 'a16', 'a19', 'a33', 'a38']
Aj
{1: [2], 2: [4], 3: [7], 4: [1], 5: [0], 6: [6], 7: [8], 8: [5], 9: [2], 10: [3]}
S
{1: 3, 2: 4, 3: 1, 4: 1, 5: 7, 6: 9, 7: 5, 8: 9, 9: 1, 10: 1}


In [25]:
j0 = list(S.keys())[0]
Aj[j0]

[2]

In [33]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

# Collect arcs data
num_arcs = len(sorted_arc_names)
label_positions = [i + 0.5 for i in range(len(sorted_arc_names))]

# Horizontal white lines at every integer y value
for i in range(num_arcs + 1):
    fig.add_shape(type="line",
        x0=0, y0=i, x1=Tend, y1=i,
        line=dict(
            color='white',
            width=1,
        ),
    )


# Get color palette for jobs
colors = [
    "LightSkyBlue", "LightCoral", "LightGreen", "LightPink",
    "LightSalmon", "LightSteelBlue", "LightYellow", "LightGray",
    "LightGoldenRodYellow", "LightCyan"
]

# Add shapes
# j0 = list(S.keys())[0]
# fig.add_shape(type="rect",
#     x0=S[j0],
#     y0=Aj[j0][0],
#     x1=S[j0] + 1,
#     y1=Aj[j0][0] + 1,
#     line=dict(
#         color="RoyalBlue",
#         width=1,
#     ),
#     fillcolor="LightSkyBlue",
# )
# fig.update_shapes(dict(xref='x', yref='y'))

jobs = list(S.keys())
for j in jobs:
    for i in range(len(Aj[j])):
        fig.add_shape(type="rect",
            x0=S[j],
            y0=Aj[j][i],
            x1=S[j] + 1,
            y1=Aj[j][i] + 1,
            line=dict(
                color=colors[jobs.index(j) % len(colors)],
                width=1,
            ),
            fillcolor=colors[jobs.index(j) % len(colors)],
            # hovertext=f"Job {j} - Arc {sorted_arc_names[Aj[j][i]]}",
        )
        fig.update_shapes(dict(xref='x', yref='y'))


# Layout
fig.update_layout(
    width=800,
    height=600,
    xaxis=dict(
        title="Time",
        range=[0, Tend],
        tickvals=list(range(0, Tend + 1)),
    ),
    yaxis=dict(
        type="category",
        title="Arcs",
        range=[0, num_arcs],
        tickvals=label_positions,
        ticktext=sorted_arc_names,
        showgrid=False,
    ),
)


fig.show()